In [3]:
import pandas as pd
import glob
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
import ast
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split
# Step 1: Import and Concatenate Datasets
file_paths = glob.glob(r'C:\Users\spt909\Documents\Guvi\Project 3\*.xlsx')  # Replace with the correct folder path

dataframes = []
for file_path in file_paths:
    city_name = file_path.split('\\')[-1].split('.')[0]  # Assuming city name is in the file name
    df = pd.read_excel(file_path)
    df['City'] = city_name  # Add city name to each dataframe
    dataframes.append(df)

# Concatenate all dataframes into one
final_df = pd.concat(dataframes, ignore_index=True)


# Step 2: Handling Missing Values
# Handling numerical missing values
numerical_cols = final_df.select_dtypes(include=['float64', 'int64']).columns
final_df[numerical_cols] = final_df[numerical_cols].fillna(final_df[numerical_cols].mean())

# Handling categorical missing values
categorical_cols = final_df.select_dtypes(include=['object']).columns
final_df[categorical_cols] = final_df[categorical_cols].fillna(final_df[categorical_cols].mode().iloc[0])
def parse_json_column(df, column_name):
    # Convert strings that look like dictionaries to actual Python dictionaries
    df[column_name] = df[column_name].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    return df

# Step 3: Parse 'new_car_detail' column
df = parse_json_column(final_df, 'new_car_detail')

# Step 4: Define a function to extract details from 'new_car_detail'
def extract_car_data(df):
    # Create new columns by extracting fields from the 'new_car_detail' column
    df['name'] = df['new_car_detail'].apply(lambda x: x.get('model') if isinstance(x, dict) else None)
    df['owner'] = df['new_car_detail'].apply(lambda x: x.get('owner') if isinstance(x, dict) else None)
    df['year'] = df['new_car_detail'].apply(lambda x: x.get('modelYear') if isinstance(x, dict) else None)
    df['fuel'] = df['new_car_detail'].apply(lambda x: x.get('ft') if isinstance(x, dict) else None)
    df['km_driven'] = df['new_car_detail'].apply(lambda x: x.get('km') if isinstance(x, dict) else None)
    
    # Clean 'km_driven' by removing commas and converting to float
    df['km_driven'] = df['km_driven'].str.replace(',', '').astype(float)
    
    # Extract selling price details
    df['selling_price'] = df['new_car_detail'].apply(lambda x: x.get('price') if isinstance(x, dict) else None)
    
    # Remove currency symbol and commas
    df['selling_price'] = df['selling_price'].str.replace('₹ ', '').str.replace(',', '')
    
    # Check for 'Lakh' and 'Crore' values
    df['selling_price_value'] = df['selling_price'].str[-4:]

    # Handle 'Lakh'
    df.loc[df['selling_price_value'] == 'Lakh', 'selling_price'] = df.loc[df['selling_price_value'] == 'Lakh', 'selling_price'].str.replace(' Lakh', '').astype(float) * 100000
    
    # Handle 'Crore'
    df.loc[df['selling_price_value'] == 'rore', 'selling_price'] = df.loc[df['selling_price_value'] == 'rore', 'selling_price'].str.replace(' Crore', '').astype(float) * 10000000

    # Convert the column to numeric
    df['selling_price'] = pd.to_numeric(df['selling_price'], errors='coerce')
    
    return df

# Step 5: Apply the extraction logic
final_df = extract_car_data(final_df)

nominal_cols = ['fuel', 'owner', 'City']  # Replace with your actual nominal columns
final_df = pd.get_dummies(final_df, columns=nominal_cols, drop_first=True)

# ordinal_cols = ['new_car_overview']  # Replace with your ordinal columns
# le = LabelEncoder()
# final_df[ordinal_cols] = final_df[ordinal_cols].apply(le.fit_transform)

scaler = MinMaxScaler()
numerical_cols = final_df.select_dtypes(include=['float64', 'int64']).columns
scaled_values = scaler.fit_transform(final_df[numerical_cols])

# Add the scaled columns with new names
for i, col in enumerate(numerical_cols):
    final_df[col ] = scaled_values[:, i]
# final_df[numerical_cols] = scaler.fit_transform(final_df[numerical_cols])
def remove_outliers(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]

# Apply the IQR method to remove outliers in numerical columns
for col in numerical_cols:
    final_df = remove_outliers(final_df, col)



print(final_df.head())
final_df.to_excel('preprocessed_data.xlsx', index=False)



                                      new_car_detail  \
0  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
1  {'it': 0, 'ft': 'Petrol', 'bt': 'SUV', 'km': '...   
2  {'it': 0, 'ft': 'Petrol', 'bt': 'Hatchback', '...   
3  {'it': 0, 'ft': 'Petrol', 'bt': 'Sedan', 'km':...   
4  {'it': 0, 'ft': 'Diesel', 'bt': 'SUV', 'km': '...   

                                    new_car_overview  \
0  {'heading': 'Car overview', 'top': [{'key': 'R...   
1  {'heading': 'Car overview', 'top': [{'key': 'R...   
2  {'heading': 'Car overview', 'top': [{'key': 'R...   
3  {'heading': 'Car overview', 'top': [{'key': 'R...   
4  {'heading': 'Car overview', 'top': [{'key': 'R...   

                                     new_car_feature  \
0  {'heading': 'Features', 'top': [{'value': 'Pow...   
1  {'heading': 'Features', 'top': [{'value': 'Pow...   
2  {'heading': 'Features', 'top': [{'value': 'Pow...   
3  {'heading': 'Features', 'top': [{'value': 'Pow...   
4  {'heading': 'Features', 'top': [{'value': '